In [1]:
import json

with open('/mnt/workspace/workgroup/lr/Lin-Chen/Open-LLaVA-NeXT-mix1M/seed_data_all.json', 'r') as file:
    data = json.load(file)

In [2]:
print(json.dumps(data[0], indent=4))

{
    "id": "000000000009",
    "image": "coco/train2017/000000000009.jpg",
    "conversations_v0": [
        {
            "image_position": "single",
            "objects": [],
            "skills": [],
            "format": "Norm",
            "question": "What types of food can be seen in the image?",
            "steps": [],
            "answer": "In the image, you can see various types of food, including broccoli, bread, meat, vegetables, and fruit. These foods are presented in colorful dishes or containers."
        },
        {
            "image_position": "single",
            "objects": [],
            "skills": [],
            "format": "Norm",
            "question": "What types of containers are used to display the food?",
            "steps": [],
            "answer": "The food in the image is displayed in brightly colored plastic trays and food bins."
        },
        {
            "image_position": "single",
            "objects": [],
            "skills": [],
      

In [3]:
docs = []
for d in data:
    if len(d['caption']) == 0:
        res = ''
        for c in d['conversations_v0']:
            res += c['question'] + ' ' + c['answer'] + ' '
        docs.append(res)
    else:
        docs.append(d['caption'][1]['value'])

print(len(data))
print(len(docs))

133939
133939


In [6]:
# persona数据topk
import json
import os.path as osp
import os
import jsonlines
docs = []
with open("/mnt/workspace/code/self_evolve_mllm/evolution-code/datasets/persona.jsonl", "r") as file:
    for d in jsonlines.Reader(file):
        docs.append(d['persona'])
    
len(docs)

200000

In [11]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import MaximalMarginalRelevance

embedding_model = SentenceTransformer("/mnt/workspace/code/self_evolve_mllm/sentence-transformers/all-MiniLM-L6-v2")
umap_model = UMAP(n_neighbors=50, n_components=5, min_dist=0.0, metric='cosine')
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', prediction_data=True)
representation_model = MaximalMarginalRelevance(diversity=0.5)

vectorizer_model = CountVectorizer(stop_words="english")
ctfidf_model = ClassTfidfTransformer()
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,              
    hdbscan_model=hdbscan_model,        
    vectorizer_model=vectorizer_model,  
    ctfidf_model=ctfidf_model,          
    representation_model=representation_model,                     
    nr_topics='none',
    top_n_words = 10
)

topics, probs = topic_model.fit_transform(docs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

KeyError: 'topics_from'

In [5]:
topic_model.get_topic_info()

Topic  Count                                           Name  \
0      -1  46322                -1_scene_blue_background_street   
1       0  86339                        0_scene_blue_people_man   
2       1    219                  1_scissors_paper_tools_sewing   
3       2    121                   2_parking_meter_spaces_coins   
4       3     86                      3_bin_recycling_bags_cans   
5       4     54           4_reading_bedtime_cognitive_benefits   
6       5     48       5_tricks_skateboarders_perform_maneuvers   
7       6     45              6_boat_precautions_weather_ensure   
8       7     38         7_kitchen_contamination_hygiene_gloves   
9       8     36              8_bus_reflection_mirrors_rearview   
10      9     35              9_control_devices_buttons_remotes   
11     10     35          10_graffiti_vandalism_property_public   
12     11     31                11_tags_livestock_ears_tracking   
13     12     27        12_bulldog_skateboarding_owner_activity   
14     13     25                 13_puppy_toddler_bathtub_potty   
15     14     25                14_isotopes_atomic_text_element   
16     15     25             15_donuts_diet_diabetes_moderation   
17     16     23                        16_oven_hot_burns_mitts   
18     17     23                    17_blister_packs_bayer_text   
19     18     22           18_flooding_spraying_damage_property   
20     19     22                  19_bed_children_falling_risks   
21     20     20                   20_album_sheet_piano_vivaldi   
22     21     19                21_conveyor_donuts_belt_machine   
23     22     19      22_seaplanes_pontoons_conventional_rescue   
24     23     19              23_muscles_pelvis_structure_human   
25     24     19        24_sidewalk_electronic_recycling_broken   
26     25     18          25_bees_honeycomb_mutualism_symbiotic   
27     26     18               26_landing_gear_runway_retracted   
28     27     18            27_street_sign_foreign_multilingual   
29     28     16                    28_bed_bunk_towel_contorted   
30     29     16            29_knife_precautions_handling_child   
31     30     16            30_tree_insects_apples_hummingbirds   
32     31     14             31_toddler_children_habits_noodles   
33     32     13                  32_machine_card_keypad_screen   
34     33     12              33_archery_target_arrows_tutorial   
35     34     12       34_radiator_thermostat_temperature_dials   
36     35     12               35_bed_belongings_laundry_drying   
37     36     12              36_missing_backlit_sony_keyboards   
38     37     11       37_ribbon_ceremony_inauguration_scissors   
39     38     11              38_drying_brush_hairdresser_frizz   
40     39     11                    39_shearing_wool_helps_skin   
41     40     11  40_floodwaters_risks_infrastructure_residents   
42     41     11       41_children_empathy_sheep_responsibility   
43     42     10            42_microwaves_mug_inside_electrical   

                                       Representation  \
0   [scene, blue, background, street, table, setti...   
1   [scene, blue, people, man, background, sky, st...   
2   [scissors, paper, tools, sewing, blades, craft...   
3   [parking, meter, spaces, coins, fees, fines, d...   
4   [bin, recycling, bags, cans, park, litter, gre...   
5   [reading, bedtime, cognitive, benefits, skills...   
6   [tricks, skateboarders, perform, maneuvers, ha...   
7   [boat, precautions, weather, ensure, jackets, ...   
8   [kitchen, contamination, hygiene, gloves, uten...   
9   [bus, reflection, mirrors, rearview, convex, b...   
10  [control, devices, buttons, remotes, channels,...   
11  [graffiti, vandalism, property, public, percep...   
12  [tags, livestock, ears, tracking, traceability...   
13  [bulldog, skateboarding, owner, activity, leas...   
14  [puppy, toddler, bathtub, potty, children, sha...   
15  [isotopes, atomic, text, element, diagram, ele...   
16  [donuts, diet, 

In [6]:
updated_topic_model = topic_model.reduce_topics(docs, nr_topics=25)

In [7]:
topic_model.get_topic_info()

Topic  Count                                     Name  \
0      -1  46322               -1_scene_blue_man_captures   
1       0  86468                  0_scene_blue_people_man   
2       1    230            1_scissors_paper_tools_sewing   
3       2    121           2_parking_meter_spaces_expired   
4       3    104     3_reading_bedtime_activity_cognitive   
5       4     77      4_oven_handling_precautions_cutting   
6       5     75          5_tricks_ramp_skateboarders_dog   
7       6     55              6_pills_blister_cover_sheet   
8       7     54     7_television_control_devices_buttons   
9       8     53       8_graffiti_vandalism_property_sign   
10      9     45    9_boat_precautions_weather_conditions   
11     10     44         10_doughnuts_conveyor_belt_fryer   
12     11     39        11_toddler_children_puppy_bathtub   
13     12     37        12_landing_gear_seaplane_pontoons   
14     13     36           13_bus_reflection_view_mirrors   
15     14     33  14_hydrant_damage_spraying_consequences   
16     15     31          15_tags_livestock_ears_tracking   
17     16     25          16_isotopes_atomic_element_mass   
18     17     19         17_joint_muscles_femur_structure   
19     18     16      18_tree_insects_apples_hummingbirds   
20     19     12            19_mouse_missing_sony_backlit   
21     20     11              20_shearing_wool_helps_skin   
22     21     11                21_drying_brush_using_dog   
23     22     11      22_animals_children_empathy_develop   
24     23     10        23_microwaves_mug_electrical_oven   

                                       Representation  \
0   [scene, blue, man, captures, background, stree...   
1   [scene, blue, people, man, captures, backgroun...   
2   [scissors, paper, tools, sewing, blades, items...   
3   [parking, meter, spaces, expired, coins, city,...   
4   [reading, bedtime, activity, cognitive, relaxa...   
5   [oven, handling, precautions, cutting, burns, ...   
6   [tricks, ramp, skateboarders, dog, riding, hal...   
7   [pills, blister, cover, sheet, radiator, therm...   
8   [television, control, devices, buttons, remote...   
9   [graffiti, vandalism, property, sign, presence...   
10  [boat, precautions, weather, conditions, equip...   
11  [doughnuts, conveyor, belt, fryer, glaze, qual...   
12  [toddler, children, puppy, bathtub, sitting, p...   
13  [landing, gear, seaplane, pontoons, floats, co...   
14  [bus, reflection, view, mirrors, rearview, con...   
15  [hydrant, damage, spraying, consequences, emer...   
16  [tags, livestock, ears, tracking, health, trac...   
17  [isotopes, atomic, element, mass, diagram, ele...   
18  [joint, muscles, femur, structure, ligament, a...   
19  [tree, insects, apples, hummingbirds, seeds, f...   
20  [mouse, missing, sony, backlit, keyboards, man...   
21  [shearing, wool, helps, skin, overheating, inf...   
22  [drying, brush, using, dog, hairdresser, frizz...   
23  [animals, children, empathy, develop, responsi...   
24  [microwaves, mug, electrical, oven, heated, sa...   

                                  Representative_Docs  
0   [The image captures a moment on a city street ...  
1   [The image captures a tranquil scene on a cobb...  
2   [What is the color of the scissors in the imag...  
3   [What are the colors of the parking meter in t...  
4   [What is the person doing in the image? The pe...  
5   [What are the necessary precautions to maintai...  
6   [What is the main activity happening in the im...  
7   [The image presents a gray rectangular blister...  
8   [How many remote controls are visible in the i...  
9   [What is the main subject of the image that ha...  
10  [What is the name of the boat in the image? Th...  
11  [What is happening inside the doughnut shop? I...  
12  [What is the position of the little boy in the...  
13  [Is the airplane's landing gear down or up? Th...  
14  [What is the primary object visible in the mir...  
15  [Is the fire hydrant functioning p

In [8]:
topic_model.visualize_barchart()